In [1]:
#basic script
#1. use category key (i.e. wine type) to select wines
#2. use requests to pull all wine data in json format (in batches of 100)
#3. use 'wine ID' to scrape html for tasting notes for each batch
#4.1. create new data frame with ID and tasting notes?
#4.2. directly append dat as new column
#5. append batch of 100 to rest of database
#6. save database as some file type (json?)

In [2]:
import requests as rq
import html5lib as h5

In [3]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [4]:
from html.parser import HTMLParser

In [ ]:
#enter your personal wine.com api key as text here:
my_api = 

In [5]:
#category (cats) request
#http://services.wine.com/api/beta2/service.svc/JSON/categorymap?apikey=bd46ee42ce188d3110a1fa31fd8fd567&filter=categories(490)
#payload = {'key1':'value1', 'key2':'value2'}
payload = {'apikey':my_api,
           'filter':'categories(490)'}
cats = rq.get('http://services.wine.com/api/beta2/service.svc/JSON/categorymap?',
           params=payload)

#read the json file as cats_data, i.e. categories data
cats_data = pd.read_json(cats.text,typ='series')
#normalize the json file and check
cats_norm = json_normalize(cats_data.Categories)

#unpack cats_norm further based on the Refinements column
wine_cats = pd.DataFrame()
for i in range(0,len(cats_norm)-1):
    temp = json_normalize(cats_norm.Refinements[i])
    temp.Description = cats_norm.Name[i]
    wine_cats = wine_cats.append(temp,ignore_index=True)

#wine_cats now has a list/df of all the category Ids and Names
#this will be useful for refining the catalog queries
wine_cats

,Description,Id,Name,Url
0,Wine Type,124,Red Wine,
1,Wine Type,125,White Wine,
2,Wine Type,123,Champagne & Sparkling,
3,Wine Type,126,Rosé Wine,
4,Wine Type,128,"Dessert, Sherry & Port",
5,Wine Type,134,Saké,
6,Varietal,139,Cabernet Sauvignon,
7,Varietal,140,Chardonnay,
8,Varietal,151,Sauvignon Blanc,
9,Varietal,194,Pinot Gris/Grigio,


In [ ]:
cats.url

In [6]:
#function to use requests to get all/some wines
def get_wine(category, apikey, num_calls=-1):
    

    #take input and convert to string for requests
    #these two are the same for every call to requests
    filter_string = 'categories'+'('+str(category)+')'
        
    #intitialize output dataframe
    wine_data_norm = pd.DataFrame()

    #initialize keep_iterating check
    keep_iterating = True
    if num_calls == 0: #sanity check on input
        keep_iterating = False
    #intitalize iterations counter
    i = 0

    #initialize offset
    offset = 0
    #dummy value for num_entries
    num_entries = 100
    
    #may as well get all the wine entries
    while keep_iterating:
        
        #setup size to account for reaching end of entries
        size = min(100, num_entries-offset)
    
        #setup payload and run request
        payload = {'apikey':apikey,
                   'size':size,
                   'offset':offset,
                   'filter':filter_string}
           
        wine = rq.get('http://services.wine.com/api/beta2/service.svc/JSON/catalog?',
                       params=payload)
    
        #post-processing of requested data
        wine_data = pd.read_json(wine.text)
        #appending new data
        wine_data_norm = wine_data_norm.append(json_normalize(wine_data.Products.List),ignore_index = True)
        
        #cleaning up loop variables

        #updating offset for next loop
        offset += size
        #checking number of entries
        num_entries = wine_data.Products.Total
        total_calls = round(num_entries/size) + 1
                
        #increase i before next loop
        i += 1   
        print(i, "call out of", total_calls, "completed.")
        
        #check offset and num_entries first
        if offset < num_entries:
            if num_calls < 0:
                keep_iterating = True
            elif i < num_calls:
                keep_iterating = True
            else:
                keep_iterating = False
        else:
            keep_iterating = False
        
            
    return wine_data_norm

In [7]:
#use requests to get wine html files with ID
#http://wine.com/v6/<Id>.notes

#function to get winemakers tasting notes based on wine Id
def get_notes(wine_id):
    
    #wine_id = wine_data_norm.Id[i]
    url_id = str(wine_id)
    url_base = 'http://wine.com/v6/'
    url_tag = '.notes'
    url_full = url_base + url_id + url_tag

    notes = rq.get(url_full)
    try:
        wine_notes = pd.read_html(notes.text, match='Winemaker')[0][0][0]
    except ValueError:
        wine_notes = 'NaN'

    return wine_notes
#return a string with notes or 'NaN'


In [8]:
#get all red wines category 124
wine_data = get_wine(125, my_api)

1 call out of 324 completed.
2 call out of 324 completed.
3 call out of 324 completed.
4 call out of 324 completed.
5 call out of 324 completed.
6 call out of 324 completed.
7 call out of 324 completed.
8 call out of 324 completed.
9 call out of 324 completed.
10 call out of 324 completed.
11 call out of 324 completed.
12 call out of 324 completed.
13 call out of 324 completed.
14 call out of 324 completed.
15 call out of 324 completed.
16 call out of 324 completed.
17 call out of 324 completed.
18 call out of 324 completed.
19 call out of 324 completed.
20 call out of 324 completed.
21 call out of 324 completed.
22 call out of 324 completed.
23 call out of 324 completed.
24 call out of 324 completed.
25 call out of 324 completed.
26 call out of 324 completed.
27 call out of 324 completed.
28 call out of 324 completed.
29 call out of 324 completed.
30 call out of 324 completed.
31 call out of 324 completed.
32 call out of 324 completed.
33 call out of 324 completed.
34 call out of 324 

In [10]:
wine_data

,Appellation.Id,Appellation.Name,Appellation.Region.Area,Appellation.Region.Id,Appellation.Region.Name,Appellation.Region.Url,Appellation.Url,Community.Reviews.HighestScore,Community.Reviews.List,Community.Reviews.Url,...,Varietal.WineType.Url,Vineyard.GeoLocation.Latitude,Vineyard.GeoLocation.Longitude,Vineyard.GeoLocation.Url,Vineyard.Id,Vineyard.ImageUrl,Vineyard.Name,Vineyard.Url,Vintage,Vintages.List
0,2454,Carneros,None,101,California,http://www.wine.com/v6/California/wine/list.as...,http://www.wine.com/v6/Carneros/wine/list.aspx...,10,[],http://www.wine.com/v6/Rombauer-Chardonnay-201...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,http://www.wine.com/v6/aboutwine/mapof.aspx?wi...,8337,http://cache.wine.com/aboutwine/basics/images/...,Rombauer Vineyards,http://www.wine.com/v6/Rombauer-Vineyards/lear...,2015,[]
1,2398,Napa Valley,None,101,California,http://www.wine.com/v6/California/wine/list.as...,http://www.wine.com/v6/Napa-Valley/wine/list.a...,10,[],http://www.wine.com/v6/Mayacamas-Chardonnay-20...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,http://www.wine.com/v6/aboutwine/mapof.aspx?wi...,8466,http://cache.wine.com/aboutwine/basics/images/...,Mayacamas,http://www.wine.com/v6/Mayacamas/learnabout.as...,2014,[]
2,2398,Napa Valley,None,101,California,http://www.wine.com/v6/California/wine/list.as...,http://www.wine.com/v6/Napa-Valley/wine/list.a...,0,[],http://www.wine.com/v6/Cakebread-Chardonnay-20...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,http://www.wine.com/v6/aboutwine/mapof.aspx?wi...,8518,http://cache.wine.com/aboutwine/basics/images/...,Cakebread,http://www.wine.com/v6/Cakebread/learnabout.as...,2015,[]
3,2454,Carneros,None,101,California,http://www.wine.com/v6/California/wine/list.as...,http://www.wine.com/v6/Carneros/wine/list.aspx...,10,[],http://www.wine.com/v6/Shafer-Red-Shoulder-Ran...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,http://www.wine.com/v6/aboutwine/mapof.aspx?wi...,6191,http://cache.wine.com/aboutwine/basics/images/...,Shafer Vineyards,http://www.wine.com/v6/Shafer-Vineyards/learna...,2014,[]
4,2462,Central Coast,None,101,California,http://www.wine.com/v6/California/wine/list.as...,http://www.wine.com/v6/Central-Coast/wine/list...,10,[],http://www.wine.com/v6/Mer-Soleil-Santa-Lucia-...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,,8627,http://cache.wine.com/aboutwine/basics/images/...,Mer Soleil,http://www.wine.com/v6/Mer-Soleil/learnabout.a...,2014,[]
5,2371,Sonoma County,None,101,California,http://www.wine.com/v6/California/wine/list.as...,http://www.wine.com/v6/Sonoma-County/wine/list...,10,[],http://www.wine.com/v6/Carneros-Hills-Winery-C...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,,999995206,http://cache.wine.com/aboutwine/basics/images/...,Carneros Hills Winery,http://www.wine.com/v6/Carneros-Hills-Winery/l...,2013,[]
6,10071,Marlborough,None,114,New Zealand,http://www.wine.com/v6/New-Zealand/wine/list.a...,http://www.wine.com/v6/Marlborough/wine/list.a...,10,[],http://www.wine.com/v6/Forrest-Estate-Sauvigno...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,,9620,http://cache.wine.com/aboutwine/basics/images/...,Forrest Estate,http://www.wine.com/v6/Forrest-Estate/learnabo...,2015,[]
7,2371,Sonoma County,None,101,California,http://www.wine.com/v6/California/wine/list.as...,http://www.wine.com/v6/Sonoma-County/wine/list...,10,[],http://www.wine.com/v6/Kendall-Jackson-Vintner...,...,http://www.wine.com/v6/White-Wines/wine/list.a...,-360,-360,http://www.wine.com/v6/aboutwine/mapof.aspx?wi...,8788,http://cache.wine.com/aboutwine/basics/images/...,Kendall-Jackson,http://www.wine.com/v6/Kendall-Jackson/learnab...,2015,[]
8,10070,Martinborough,None,114,New Zealand,http://www.wine.com/v6/New-Zealand/wine/list.a...,http://www.wine.com/v6/Martinborough/wine/list...,10,[],http://www.wine.com/v6/Craggy-Range-Te-Muna-Ro...,...,http://www.wine.com/v6/White-Wine

In [9]:
json_output = wine_data.to_json(orient="columns")
f = open('wine_data_raw.json','w')
f.write(json_output)
f.close()

In [ ]:
wine_data[['Id','Name']]

In [ ]:
wine_data['Varietal.Name'].value_counts()

In [ ]:
# run get_notes for all wines with IDs and return a dataframe with IDs and noted
tasting_notes = pd.DataFrame()
num_wines = len(wine_data['Id'])

for i in range(0,num_wines):
    if i % round(num_wines/100) == 0:
        print(round(i/num_wines * 100 , 1), '%')
    notes_i = pd.DataFrame({'Id':[wine_data.Id[i]],
                            'Notes':[get_notes(wine_data.Id[i])]})
    tasting_notes = tasting_notes.append(notes_i, ignore_index = True)



In [ ]:
tasting_notes

In [ ]:
get_notes(white_wines.Id[27294])

In [ ]:
for i in range(27294,len(white_wines['Id'])):
    if i % 316 == 0:
        print(round(i/len(white_wines['Id']) * 100 , 1), '%')
    notes_i = pd.DataFrame({'Id':[white_wines.Id[i]],
                            'Notes':[get_notes(white_wines.Id[i])]})
    tasting_notes = tasting_notes.append(notes_i, ignore_index = True)

In [ ]:
len(white_wines['Id'])

In [ ]:
#merge dataframes on Id
wine_data_notes = pd.merge(wine_data, tasting_notes, how='left', on='Id')

In [ ]:
json_output = wine_data_notes.to_json(orient="columns")

In [ ]:
f = open('wine_notes_raw.json','w')
f.write(json_output)
f.close()